In [30]:
import pandas as pd

In [31]:
#Reading Dataframe
df = pd.read_csv('data/trades.csv')

In [61]:
#Combining both types of emissions
df.replace(['Emission - Venue A', 'Emission - Venue B'], 'Emissions', inplace=True)

In [77]:
def ohlc_resampling(df, begin, end, product, freq):
    #Turning the datetime column into the index
    df = df.set_index(pd.to_datetime(df['TradeDateTime'], infer_datetime_format=True)).drop('TradeDateTime', axis=1)
    
    #Filtering the df based on begin, end and product conditions
    condition_1 = df.index.to_series().between(begin, end, inclusive=True)
    condition_2 = df.Product == product
    
    filtered_df = df[condition_1 & condition_2]
    
    #Resampling to defined freq and grouping to get Max, Min, Open and Close
    ohlc_df = filtered_df.groupby(['Contract', pd.Grouper(freq=freq, label='left')]).\
    agg(Open = ('Price', 'first'),
        High = ('Price', 'max'),
        Low = ('Price', 'min'),
        Close = ('Price', 'last'),
        Volume = ('Quantity', 'sum')).reset_index().sort_values(['TradeDateTime', 'Contract'])

    return(ohlc_df)

In [78]:
begin = '2022-04-20 07:00'
end = '2022-04-20 13:00'
product = 'Energy'
freq = '15MIN'

ohlc_resampling(df, begin, end, product, freq)

,Contract,TradeDateTime,Open,High,Low,Close,Volume
0,Q01,2022-04-20 07:00:00,24.0725,24.0725,23.9225,24.0450,591
1,Q01,2022-04-20 07:15:00,24.0400,24.0625,23.9825,24.0350,83
2,Q01,2022-04-20 07:30:00,24.0525,24.1175,24.0450,24.0700,169
3,Q01,2022-04-20 07:45:00,24.0725,24.1225,24.0250,24.0375,311
4,Q01,2022-04-20 08:00:00,24.0375,24.1175,24.0350,24.1100,395
5,Q01,2022-04-20 08:15:00,24.0975,24.1600,24.0550,24.1525,360
6,Q01,2022-04-20 08:30:00,24.1450,24.2200,24.0675,24.1725,562
7,Q01,2022-04-20 08:45:00,24.1750,24.1750,24.1550,24.1625,41
8,Q01,2022-04-20 10:30:00,24.3750,24.3825,24.3600,24.3725,71
9,Q01,2022-04-20 10:45:00,24.3600,24.3750,24.3450,24.3525,170


In [85]:
begin = '2022-04-20 07:00'
end = '2022-04-21 17:00'
product = 'Emissions'
freq = '1D'

ohlc_resampling(df, begin, end, product, freq)

,Contract,TradeDateTime,Open,High,Low,Close,Volume
0,DA,2022-04-20,18.51250,19.26250,18.40000,18.8750,115576
2,M01,2022-04-20,18.36000,18.98500,18.09500,18.6975,8273
1,DA,2022-04-21,18.81250,19.22500,18.11875,18.6375,128895
3,M01,2022-04-21,18.98625,18.98625,17.84750,18.3375,8681
